In [184]:
import sys
import importlib
import numpy as np
u_path = "../lib"
if not u_path in sys.path:
    sys.path.append(u_path)
import autoloader
importlib.reload(autoloader)
al = autoloader.Autoloader()

In [4]:
d13data = al.f_fetch_as_iter(2020, 13)


In [253]:
tdata1 = """939
7,13,x,x,59,x,31,19"""
tdata1 = tdata1.splitlines()
tdata2 = """939
2,3,5,7"""
tdata2 = tdata2.splitlines()
tdata3 = """939
2,3,5,7,x,19"""
tdata3 = tdata3.splitlines()

I think this is an application of the CRT.
let x be the solution.
x mod a0 = 0
x mod a1 = 1
....
x mod an = n - number of skipped busses

so the bus IDs are probably relatively prime, the position on the list is the remainder (a_k)

M_k is the product of all bus IDs except for the kth
y_k satisfies M_k y_k \congruent 1 (mod m_k)

x = a_1 M_1 y_1 + ... + a_n M_n y_n


In [245]:
class d13():
    def __init__(self, sched):
        self.time = int(sched[0])
        self.bus_list = {}
        for ii, bus in enumerate(sched[1].split(',')):
            if bus == 'x':
                continue
            self.bus_list[int(bus)] = int(bus) - (ii%int(bus))

        
    def find_first_bus(self):
        cur_bus_status = {bus: bus - self.time%bus for bus in self.bus_list}
        first_bus = min(cur_bus_status, key=cur_bus_status.get)
        return first_bus * cur_bus_status[first_bus]
    
    def find_part2_sol(self):
        # bus ID is the relative prime, offset is the remainder
        sched = np.array([(p,self.bus_list[p]) for p in self.bus_list], dtype=int)
        return CRT(sched)



def CRT(p_r_pairs) -> int:
    from itertools import repeat
    """ 
    Function takes a numpy array of relatively prime numbers and the remainders 
    and uses Chinese Remainder Theorem to find the solve the system of linear congruences 

    in: p_r_pairs : numpy array in form:
    ((p0, r0)
     (p1, r1)
     ...
     (pn, rn))

    out: x given
    X = r0 M0 y0 + ... + rn Mn yn = x (mod M)

    where: 
    Mk = M / pk, 
    yk = pow(Mk, -1, pk) 
        (the inverse modulo) 
    M = prod(p0, ..., pn)

    """

    M = np.prod(p_r_pairs[:,0], dtype=np.int64)
    Mk = np.divide(M, p_r_pairs[:,0]).astype(np.int64)
    yk = np.fromiter(map(pow, map(int, Mk),repeat(-1),map(int,p_r_pairs[:,0])), dtype=np.int)
    X = np.sum(p_r_pairs[:,1] * Mk * yk)
    return X % M



            

In [254]:
test1 = d13(tdata1)
t1ans = test.find_part2_sol()
test2 = d13(tdata2)
t2ans = test2.find_part2_sol()
test3 = d13(tdata3)
t3ans = test3.find_part2_sol()
print(t3ans/t2ans)
print(t1ans, t2ans, t3ans)



11.632911392405063
1058443396696792 158 1838


In [179]:
test = d13(d13data)
print(test.bus_list)
print(test.find_part2_sol())



{19: 19, 41: 32, 823: 804, 23: 19, 17: 15, 29: 10, 443: 393, 37: 18, 13: 2}
1058443396696792


In [168]:
a = np.array(((3,2),(5,3),(7,2)))
ans = 1068781
31 - (ans % 31)

6

In [160]:
test = d13(d13data)
print(test.find_first_bus())



2215


Day 14
------

In [216]:
d14data = al.f_fetch_as_iter(2020, 14)


In [224]:
d14tdata1 = """mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0"""
d14tdata1 = d14tdata1.splitlines()
d14tdata2 = """mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1"""
d14tdata2 = d14tdata2.splitlines()

In [220]:
def line_parse(line):
    n, _, val = line.split()
    return n, val
def val_to_36bitstring(val):
    val = int(val)
    bitstring = [0]*36
    while val > 0:
        place = np.floor(np.log(val)/np.log(2)).astype(int)
        bitstring[place] = 1
        val -= 2**place
    return bitstring
def bitstring_to_val(bitstring):
    val = 0
    for place, dig in enumerate(bitstring):
        val += (2**place)*dig
    return val


class space_dock():
    def __init__(self, bit_mask):
        self.set_mask(bit_mask)
        self.mem = dict()
    def set_mask(self, bit_mask):
        self.bit_mask = []
        for bit in bit_mask[::-1]:
            self.bit_mask.append(bit)
        
    def apply_mask(self, val):
        for ii, bit in enumerate(self.bit_mask):
            if bit == 'X':
                continue
            val[ii] = int(bit)
        return val
    def store_mem(self, loc, val):
        val = val_to_36bitstring(val)
        #print("unmask val: ", [str(v) for v in val])
        val = self.apply_mask(val)
        #print("  mask    : ", str(self.bit_mask))
        #print("  mask val: ", [str(v) for v in val])
        self.mem[int(loc)] = val
    def __abs__(self):
        tot = 0
        for loc in self.mem:
            tot += bitstring_to_val(self.mem[loc])
        return tot



def d14(data, version=space_dock):
    _, mask = line_parse(data[0])
    sd = version(mask)
    for line in data[1:]:
        name, val = line_parse(line)
        if name == 'mask':
            sd.set_mask(val)
        else:
            _, loc = name.split('[')
            loc = loc[:-1]
            sd.store_mem(loc, val)
    print(abs(sd))


d14(d14tdata1)
        

165


In [218]:
d14(d14data)

14553106347726


In [243]:
def mem_decode(mem_code):
    pos_mem = [[]]
    for place in mem_code:
        if place == 2:
            n_mem = []
            for mem in pos_mem:
                mem0 = mem.copy()
                mem1 = mem.copy()
                mem0.append(0)
                mem1.append(1)
                n_mem.append(mem0)
                n_mem.append(mem1)
            pos_mem = n_mem
        else:
            for mem in pos_mem:
                mem.append(place)
    return pos_mem

class space_dock_v2(space_dock):
    def store_mem(self, loc, val):
        val = val_to_36bitstring(val)
        loc = val_to_36bitstring(loc)
        loc = self.apply_mask(loc)
        locs = mem_decode(loc)
        for loc in locs:
            adr = bitstring_to_val(loc)
            self.mem[adr] = val


    def apply_mask(self, val):
        for ii, bit in enumerate(self.bit_mask):
            if bit == 'X':
                val[ii] = 2
            if bit == '0':
                continue
            if bit == '1':
                val[ii] = 1
        return val
        
d14(d14data, space_dock_v2)

2737766154126
